In [53]:
from telethon.sync import TelegramClient
 
import csv
 
from telethon.tl.functions.messages import GetDialogsRequest
from telethon.tl.types import InputPeerEmpty
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.types import PeerChannel
import json

In [9]:
with open("tg_cfg.json", "r") as f:
    cfg = json.load(f)
app_id = cfg['app_id']
api_hash = cfg['api_hash']
phone = cfg['phone']
    

In [10]:
client = TelegramClient(phone, app_id, api_hash)

In [21]:
client = await client.start()

In [31]:
chats = []
last_date = None
size_chats = 200
groups=[]

In [32]:
result = client(GetDialogsRequest(
            offset_date=last_date,
            offset_id=0,
            offset_peer=InputPeerEmpty(),
            limit=size_chats,
            hash = 0
        ))
result = await result
chats.extend(result.chats)

In [36]:
for chat in chats:
   try:
       if chat.megagroup== True:
           groups.append(chat)
   except:
       continue

In [40]:
valid_titles = {"[a23] Машинное обучение (ML)", "[a23] Годовой проект", "[a23] Математика для анализа данных", 
               "[a23] Промышленная разработка", "[a2*] Прикладной MLOps (Владислав Гончаренко)", "[a23] Инструменты разработки", 
               "[a23] Python", }
valid_groups = []
for g in groups:
    if g.title in valid_titles:
        valid_groups.append(g)

In [70]:
all_participants = []

for group in valid_groups:
    group_users = await client.get_participants(group)
    for user in group_users:
        if user.username:
           username= user.username
        else:
           username= ""
        if user.first_name:
           first_name= user.first_name
        else:
           first_name= ""
        if user.last_name:
           last_name= user.last_name
        else:
           last_name= ""
        user_id = user.id
        name = (first_name + ' ' + last_name).strip()
        all_participants.append([user_id, username, name, group.title])

In [90]:
import pandas as pd

df = pd.DataFrame(data=all_participants, columns=['user_id', 'user_name', 'name', 'group_name'])
df.to_csv("../data/mlds_chats_users.csv", index=False)

In [59]:
unique_groups = df['group_name'].unique()

In [63]:
from tqdm import tqdm

In [64]:
mlds_chats = []
for g in tqdm(valid_groups):
    all_messages = []
    offset_id = 0
    limit = 100
    total_messages = 0
    total_count_limit = 0
    while True:
        history = client(GetHistoryRequest(
           peer=g,
           offset_id=offset_id,
           offset_date=None,
           add_offset=0,
           limit=limit,
           max_id=0,
           min_id=0,
           hash=0
        ))
        history = await history
        if not history.messages:
           break
        messages = history.messages
        for message in messages:
           all_messages.append(message.to_dict())
        offset_id = messages[len(messages) - 1].id
        if total_count_limit != 0 and total_messages >= total_count_limit:
           break
    mlds_chats.append([all_messages, g.title])

100%|█████████████████████████████████████████████| 7/7 [01:05<00:00,  9.29s/it]


In [103]:
users_navig_dict = dict()

for _, row in df.iterrows():
    user_id = row.user_id
    user_name = row.user_name
    name = row['name']
    users_navig_dict.update({user_id: [user_name, name]})

In [330]:
full_chats_history = []


for group in mlds_chats:
    msgs, chat_name = group
    for msg in msgs:
        if msg['_'] != 'Message':
            continue

        from_id = msg['from_id']['user_id']
        try:
            from_username = users_navig_dict[from_id][0]
        except KeyError:
            continue
        from_name = users_navig_dict[from_id][1]

        
        msg_id = msg['id']
        msg_text = msg['message']
        
        
        group_id = msg['peer_id']['channel_id']

        msg_date = msg['date']
        
        datetime_obj = datetime.combine(msg_date.date(), datetime.min.time())
        day_timestamp = datetime_obj.timestamp()
        year = msg_date.year
        month = msg_date.month
        day = msg_date.day
        hour = msg_date.hour
        minute = msg_date.minute
        second = msg_date.second
        
        full_chats_history.append([msg_id, from_id, from_username, from_name, group_id, chat_name, msg_text, day_timestamp, year, month, day, hour, minute, second])

In [331]:
columns = ['msg_id', 'from_id', 'from_username', 'from_name', 'group_id', 'chat_name', 'msg_text', 'day_timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second']
df_msgs = pd.DataFrame(data=full_chats_history, columns=columns)

In [332]:
df_msgs.to_csv("../data/mlds_chats_msgs.csv")